## 图像数据读取和处理

### 从Github下载图片数据

In [ ]:
!git clone https://github.com/Zwysun/CNN.git


Cloning into 'CNN'...
remote: Enumerating objects: 643, done.
remote: Counting objects: 100% (643/643), done.
remote: Compressing objects: 100% (503/503), done.
remote: Total 643 (delta 138), reused 640 (delta 138), pack-reused 0
Receiving objects: 100% (643/643), 31.28 MiB | 30.48 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [ ]:
!ls

CNN  sample_data


### 读取数据

## 加载预训练模型（DenseNet）

In [ ]:
import numpy as np
import pandas as pd
from torch import nn
import torchvision.models as models

densenet = models.densenet161(pretrained=True)

densenet.classifier = nn.Linear(in_features=2208, out_features=19, bias=True)  # 输出层参数修改
densenet.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) # 输入参数修改
print(densenet)


In [ ]:
def trainandsave(): 
    trainloader = loadtraindata() # 神经网络结构
    net = Net()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 学习率为0.001 criterion = nn.CrossEntropyLoss() # 损失函数也可以自己定义，我们这里用的交叉熵损失函数 # 训练部分
    
    for epoch in range(5): # 训练的数据量为5个epoch，每个epoch为一个循环
                            # 每个epoch要训练所有的图片，每训练完成200张便打印一下训练的效果（loss值）
        running_loss = 0.0  # 定义一个变量方便我们对loss进行输出
        for i, data in enumerate(trainloader, 0): # 这里我们遇到了第一步中出现的trailoader，代码传入数据
            # enumerate是python的内置函数，既获得索引也获得数据
            # get the inputs
            inputs, labels = data # data是从enumerate返回的data，包含数据和标签信息，分别赋值给inputs和labels

            # wrap them in Variable
            inputs, labels = Variable(inputs), Variable(labels) # 转换数据格式用Variable
           
            optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度

            # forward + backward + optimize
            outputs = net(inputs) # 把数据输进CNN网络net
            loss = criterion(outputs, labels) # 计算损失值
            loss.backward() # loss反向传播
            optimizer.step() # 反向传播后参数更新 
            running_loss += loss.data[0]      # loss累加
            if i % 200 == 199:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200)) # 然后再除以200，就得到这两百次的平均损失值
                running_loss = 0.0  # 这一个200次结束后，就把running_loss归零，下一个200次继续使用

    print('Finished Training') # 保存神经网络
    torch.save(net, 'net.pkl') # 保存整个神经网络的结构和模型参数
    torch.save(net.state_dict(), 'net_params.pkl') # 只保存神经网络的模型参数